In [1]:
import pandas as pd
import urllib.parse

In [2]:
def consolidar_dados_temperatura():
    """
    Realiza o web scraping dos dados de temperatura de múltiplos links,
    os consolida e salva em um único arquivo CSV.
    """
    # Dicionário para mapear o nome do estado (para a nova coluna) à sua respectiva URL
    urls_estados = {
        "Acre": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/acre-TAVG-Trend.txt",
        "Alagoas": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/alagoas-TAVG-Trend.txt",
        "Amapá": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/amapa-TAVG-Trend.txt",
        "Amazonas": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/amazonas-TAVG-Trend.txt",
        "Bahia": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/bahia-TAVG-Trend.txt",
        "Ceará": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/ceara-TAVG-Trend.txt",
        "Distrito Federal": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/distrito-federal-TAVG-Trend.txt",
        "Espírito Santo": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/esp%C3%ADrito-santo-TAVG-Trend.txt",
        "Goiás": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/goias-TAVG-Trend.txt",
        "Maranhão": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/maranh%C3%A3o-TAVG-Trend.txt",
        "Mato Grosso": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/mato-grosso-TAVG-Trend.txt",
        "Mato Grosso do Sul": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/mato-grosso-do-sul-TAVG-Trend.txt",
        "Minas Gerais": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/minas-gerais-TAVG-Trend.txt",
        "Pará": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/para-TAVG-Trend.txt",
        "Paraíba": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/paraiba-TAVG-Trend.txt",
        "Paraná": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/parana-TAVG-Trend.txt",
        "Pernambuco": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/pernambuco-TAVG-Trend.txt",
        "Piauí": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/piaui-TAVG-Trend.txt",
        "Rio de Janeiro": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/rio-de-janeiro-TAVG-Trend.txt",
        "Rio Grande do Norte": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/rio-grande-do-norte-TAVG-Trend.txt",
        "Rio Grande do Sul": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/rio-grande-do-sul-TAVG-Trend.txt",
        "Rondônia": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/rond%C3%B4nia-TAVG-Trend.txt",
        "Roraima": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/roraima-TAVG-Trend.txt",
        "Santa Catarina": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/santa-catarina-TAVG-Trend.txt",
        "São Paulo": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/sao-paulo-TAVG-Trend.txt",
        "Sergipe": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/sergipe-TAVG-Trend.txt",
        "Tocantins": "https://berkeley-earth-temperature.s3.us-west-1.amazonaws.com/Regional/TAVG/tocantins-TAVG-Trend.txt"
    }
    
    # Nomes das colunas, conforme descrito nos arquivos de texto
    nomes_colunas = [
        'Ano', 'Mês', 'Anomalia_Mensal', 'Incerteza_Mensal', 'Anomalia_Anual', 
        'Incerteza_Anual', 'Anomalia_5_Anos', 'Incerteza_5_Anos', 'Anomalia_10_Anos',
        'Incerteza_10_Anos', 'Anomalia_20_Anos', 'Incerteza_20_Anos'
    ]
    
    lista_de_dataframes = []
    
    print("Iniciando a coleta de dados...")

    for estado, url in urls_estados.items():
        try:
            print(f"Processando dados de: {estado}...")
            # O pandas pode ler diretamente de uma URL
            # Usamos 'comment="%"' para ignorar as linhas de cabeçalho
            # 'delim_whitespace=True' trata um ou mais espaços como separador
            # 'header=None' pois não há uma linha de cabeçalho com os nomes
            df_estado = pd.read_csv(
                url, 
                comment='%',
                header=None,
                sep='\s+',
                encoding='latin-1'
            )
            
            # Atribui os nomes corretos às colunas
            df_estado.columns = nomes_colunas
            
            # Adiciona a coluna 'Estado' para identificar a origem dos dados
            df_estado['Estado'] = estado
            
            # Adiciona o dataframe processado à nossa lista
            lista_de_dataframes.append(df_estado)
            
        except Exception as e:
            print(f"  -> Falha ao processar dados de {estado}. Erro: {e}")
            
    if not lista_de_dataframes:
        print("Nenhum dado foi coletado. Encerrando o script.")
        return

    # Concatena todos os dataframes da lista em um único dataframe
    df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)
    
    # Reordena as colunas para que 'Estado' apareça primeiro
    colunas_ordenadas = ['Estado'] + nomes_colunas
    df_consolidado = df_consolidado[colunas_ordenadas]

    # Salva o resultado em um novo arquivo CSV
    nome_arquivo_saida = 'temperaturas_consolidadas.csv'
    df_consolidado.to_csv(nome_arquivo_saida, index=False, encoding='utf-8')
    
    print(f"\nProcesso concluído com sucesso!")
    print(f"Os dados foram consolidados e salvos no arquivo: '{nome_arquivo_saida}'")
    print(f"Total de linhas processadas: {len(df_consolidado)}")

# Executa a função
consolidar_dados_temperatura()

Iniciando a coleta de dados...
Processando dados de: Acre...
Processando dados de: Alagoas...
Processando dados de: Amapá...
  -> Falha ao processar dados de Amapá. Erro: HTTP Error 403: Forbidden
Processando dados de: Amazonas...
Processando dados de: Bahia...
Processando dados de: Ceará...
  -> Falha ao processar dados de Ceará. Erro: HTTP Error 403: Forbidden
Processando dados de: Distrito Federal...
Processando dados de: Espírito Santo...
  -> Falha ao processar dados de Espírito Santo. Erro: HTTP Error 403: Forbidden
Processando dados de: Goiás...
  -> Falha ao processar dados de Goiás. Erro: HTTP Error 403: Forbidden
Processando dados de: Maranhão...
  -> Falha ao processar dados de Maranhão. Erro: HTTP Error 403: Forbidden
Processando dados de: Mato Grosso...
Processando dados de: Mato Grosso do Sul...
Processando dados de: Minas Gerais...
Processando dados de: Pará...
  -> Falha ao processar dados de Pará. Erro: HTTP Error 403: Forbidden
Processando dados de: Paraíba...
  -> Fa